<a href="https://colab.research.google.com/github/Gaurav-phatkare/Retrieval-Augmented-Generation_RAG/blob/main/RAG_Fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sat Jul 27 16:31:50 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import torch

## building

1. Open a PDF document
2. Format the text of that PDF suitable for embedding model
3. embed into chunks and then convert it into numerical representation
4. use vector db to search relevent cuunk
5. crete prompt that incoporates the retrived peices of text
6. generate answer

## Import PDF


In [2]:
import os
import requests

pdf_path = "human-nutrition-text.pdf"

# download PDF

if not os.path.exists(pdf_path):
  print("File doesnt exists")

  url = "https://pressbooks.oer.hawaii.edu/humannutrition2/open/download?type=pdf"

  filename = pdf_path
  res = requests.get(url)

  if res.status_code == 200:
    with open(filename, "wb") as file:
      file.write(res.content)
    print(f"file has been downloaded and stored as {filename}")

  else:
    print(f"failed to download file, Status code : {res.status_code}")
else:
  print("file exists")

File doesnt exists
file has been downloaded and stored as human-nutrition-text.pdf


In [3]:
!pip install pyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 37.3 MB/s eta 0:00:00


In [4]:
!pip install tqdm

In [5]:
import fitz
from tqdm.auto import tqdm


def text_formatter(text:str):
  cleaned_text = text.replace("\n", " ").strip()
  return cleaned_text


def open_and_read_pdf(pdf_path):
  doc = fitz.open(pdf_path)

  pages_and_texts = []

  for page_number, page in tqdm(enumerate(doc)):
    text = page.get_text()

    text = text_formatter(text=text)

    pages_and_texts.append({"page_number" : page_number - 41,
                            "page_char_count" : len(text),
                            "page_word_count" : len(text.split(" ")),
                            "page_sentence_count_raw" : len(text.split(". ")),
                            "page_token_count": len(text) / 4,
                            "text" : text})

  return pages_and_texts


pages_and_texts = open_and_read_pdf(pdf_path)
pages_and_texts[:3]




0it [00:00, ?it/s]

[{'page_number': -41,
  'page_char_count': 29,
  'page_word_count': 4,
  'page_sentence_count_raw': 1,
  'page_token_count': 7.25,
  'text': 'Human Nutrition: 2020 Edition'},
 {'page_number': -40,
  'page_char_count': 0,
  'page_word_count': 1,
  'page_sentence_count_raw': 1,
  'page_token_count': 0.0,
  'text': ''},
 {'page_number': -39,
  'page_char_count': 320,
  'page_word_count': 54,
  'page_sentence_count_raw': 1,
  'page_token_count': 80.0,
  'text': 'Human Nutrition: 2020  Edition  UNIVERSITY OF HAWAI‘I AT MĀNOA  FOOD SCIENCE AND HUMAN  NUTRITION PROGRAM  ALAN TITCHENAL, SKYLAR HARA,  NOEMI ARCEO CAACBAY, WILLIAM  MEINKE-LAU, YA-YUN YANG, MARIE  KAINOA FIALKOWSKI REVILLA,  JENNIFER DRAPER, GEMADY  LANGFELDER, CHERYL GIBBY, CHYNA  NICOLE CHUN, AND ALLISON  CALABRESE'}]

In [6]:
import random

random.sample(pages_and_texts, 2)

[{'page_number': 935,
  'page_char_count': 957,
  'page_word_count': 164,
  'page_sentence_count_raw': 6,
  'page_token_count': 239.25,
  'text': 'due to the overload principle that our bodies will adapt to with  continuous repetition. For example, if you run a mile everyday for  a week, in a few weeks you would be able to run further and likely  faster.  Learning Activities  Technology Note: The second edition of the Human  Nutrition Open Educational Resource (OER) textbook  features interactive learning activities.\xa0 These activities are  available in the web-based textbook and not available in the  downloadable versions (EPUB, Digital PDF, Print_PDF, or  Open Document).  Learning activities may be used across various mobile  devices, however, for the best user experience it is strongly  recommended that users complete these activities using a  desktop or laptop computer and in Google Chrome.  \xa0 An interactive or media element has been  excluded from this version of the text. Yo

In [7]:
import pandas as pd

df = pd.DataFrame(pages_and_texts)
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text
0,-41,29,4,1,7.25,Human Nutrition: 2020 Edition
1,-40,0,1,1,0.00,
2,-39,320,54,1,80.00,Human Nutrition: 2020 Edition UNIVERSITY OF ...
3,-38,212,32,1,53.00,Human Nutrition: 2020 Edition by University of...
4,-37,797,145,2,199.25,Contents Preface University of Hawai‘i at Mā...


In [8]:
df.describe()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count
count,1208.00000,1208.000000,1208.000000,1208.000000,1208.000000
mean,562.50000,1148.594371,198.889901,9.972682,287.148593
std,348.86387,560.441673,95.747365,6.187226,140.110418
min,-41.00000,0.000000,1.000000,1.000000,0.000000
25%,260.75000,762.750000,134.000000,4.000000,190.687500
50%,562.50000,1232.500000,215.000000,10.000000,308.125000
75%,864.25000,1605.250000,271.250000,14.000000,401.312500
max,1166.00000,2308.000000,429.000000,32.000000,577.000000


In [9]:
# Text processing
from spacy.lang.en import English

nlp = English()


nlp.add_pipe('sentencizer')


doc = nlp("this is a sentence. This is another sentence... . I love tensorflow.")

# assert len(list(doc.sents)) == 3

list(doc.sents)

[this is a sentence., This is another sentence... ., I love tensorflow.]

In [10]:
for item in tqdm(pages_and_texts):
  item['sentences'] =  list(nlp(item['text']).sents)

  item['sentences'] = [str(sentence) for sentence in item['sentences']]

  item['page_sentence_count_spacy'] = len(item['sentences'])



  0%|          | 0/1208 [00:00<?, ?it/s]

In [11]:
random.sample(pages_and_texts, 1)

[{'page_number': 1144,
  'page_char_count': 1683,
  'page_word_count': 252,
  'page_sentence_count_raw': 16,
  'page_token_count': 420.75,
  'text': 'that regulate this profession.1 Go to https:/ /www.cdrnet.org/ certifications to learn more.  Working in Nutrition  Registered dietitians (RDs)/registered dietitians nutritionist (RDNs)  and nutritionists plan food and nutrition programs, promote healthy  eating habits, and recommend dietary modifications based on the  needs of individuals or groups. For example, an RD/RDN might  teach a patient with hypertension how to follow the DASH diet  and reduce their sodium intake. Nutrition-related careers can be  extremely varied. Some individuals work in the government, while  others are solely in the private sectors (i.e., private practice,  worksite wellness, hospitals, outpatient clinics, etc). Some jobs in  nutrition focus on working with elite athletes, while others provide  guidance to patients with long-term, life-threatening diseases. B

In [12]:
df = pd.DataFrame(pages_and_texts)

df.describe()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,page_sentence_count_spacy
count,1208.00000,1208.000000,1208.000000,1208.000000,1208.000000,1208.000000
mean,562.50000,1148.594371,198.889901,9.972682,287.148593,10.319536
std,348.86387,560.441673,95.747365,6.187226,140.110418,6.300843
min,-41.00000,0.000000,1.000000,1.000000,0.000000,0.000000
25%,260.75000,762.750000,134.000000,4.000000,190.687500,5.000000
50%,562.50000,1232.500000,215.000000,10.000000,308.125000,10.000000
75%,864.25000,1605.250000,271.250000,14.000000,401.312500,15.000000
max,1166.00000,2308.000000,429.000000,32.000000,577.000000,28.000000


In [13]:
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text,sentences,page_sentence_count_spacy
0,-41,29,4,1,7.25,Human Nutrition: 2020 Edition,[Human Nutrition: 2020 Edition],1
1,-40,0,1,1,0.00,,[],0
2,-39,320,54,1,80.00,Human Nutrition: 2020 Edition UNIVERSITY OF ...,[Human Nutrition: 2020 Edition UNIVERSITY OF...,1
3,-38,212,32,1,53.00,Human Nutrition: 2020 Edition by University of...,[Human Nutrition: 2020 Edition by University o...,1
4,-37,797,145,2,199.25,Contents Preface University of Hawai‘i at Mā...,[Contents Preface University of Hawai‘i at M...,2


In [14]:
chunk_size = 10

def split_list(input_list, slice_size = chunk_size):

  return [input_list[i:i+slice_size] for i in range(0, len(input_list), slice_size)]

test_list = list(range(25))

split_list(test_list)

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
 [20, 21, 22, 23, 24]]

In [15]:
for item in tqdm(pages_and_texts):
  item['sentence_chunks'] = split_list(input_list = item['sentences'], slice_size = chunk_size)

  item['num_chunks'] = len(item['sentence_chunks'])

  0%|          | 0/1208 [00:00<?, ?it/s]

In [16]:
random.sample(pages_and_texts, 1)

[{'page_number': 862,
  'page_char_count': 1309,
  'page_word_count': 215,
  'page_sentence_count_raw': 11,
  'page_token_count': 327.25,
  'text': 'There are steps that parents and caregivers can take to prevent  iron-deficiency anemia, such as adding more iron-rich foods to  a child’s diet, including lean meats, fish, poultry, eggs, legumes,  and iron-enriched whole-grain breads and cereals. A toddler’s diet  should provide 7 to 10 milligrams of iron daily. Although milk is  critical for the bone-building calcium that it provides, intake should  not exceed the RDA to avoid displacing foods rich with iron.  Children may also be given a daily supplement, using infant vitamin  drops with iron or ferrous sulfate drops. If iron-deficiency anemia  does occur, treatment includes a dosage of 3 milligrams per  kilogram once daily before breakfast, usually in the form of a ferrous  sulfate syrup. Consuming vitamin C, such as orange juice, can also  help to improve iron absorption.10  \xa0 Lear

In [17]:
df = pd.DataFrame(pages_and_texts)

df.describe()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,page_sentence_count_spacy,num_chunks
count,1208.00000,1208.000000,1208.000000,1208.000000,1208.000000,1208.000000,1208.000000
mean,562.50000,1148.594371,198.889901,9.972682,287.148593,10.319536,1.525662
std,348.86387,560.441673,95.747365,6.187226,140.110418,6.300843,0.644397
min,-41.00000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,260.75000,762.750000,134.000000,4.000000,190.687500,5.000000,1.000000
50%,562.50000,1232.500000,215.000000,10.000000,308.125000,10.000000,1.000000
75%,864.25000,1605.250000,271.250000,14.000000,401.312500,15.000000,2.000000
max,1166.00000,2308.000000,429.000000,32.000000,577.000000,28.000000,3.000000


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1208 entries, 0 to 1207
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   page_number                1208 non-null   int64  
 1   page_char_count            1208 non-null   int64  
 2   page_word_count            1208 non-null   int64  
 3   page_sentence_count_raw    1208 non-null   int64  
 4   page_token_count           1208 non-null   float64
 5   text                       1208 non-null   object 
 6   sentences                  1208 non-null   object 
 7   page_sentence_count_spacy  1208 non-null   int64  
 8   sentence_chunks            1208 non-null   object 
 9   num_chunks                 1208 non-null   int64  
dtypes: float64(1), int64(6), object(3)
memory usage: 94.5+ KB


In [19]:
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text,sentences,page_sentence_count_spacy,sentence_chunks,num_chunks
0,-41,29,4,1,7.25,Human Nutrition: 2020 Edition,[Human Nutrition: 2020 Edition],1,[[Human Nutrition: 2020 Edition]],1
1,-40,0,1,1,0.00,,[],0,[],0
2,-39,320,54,1,80.00,Human Nutrition: 2020 Edition UNIVERSITY OF ...,[Human Nutrition: 2020 Edition UNIVERSITY OF...,1,[[Human Nutrition: 2020 Edition UNIVERSITY O...,1
3,-38,212,32,1,53.00,Human Nutrition: 2020 Edition by University of...,[Human Nutrition: 2020 Edition by University o...,1,[[Human Nutrition: 2020 Edition by University ...,1
4,-37,797,145,2,199.25,Contents Preface University of Hawai‘i at Mā...,[Contents Preface University of Hawai‘i at M...,2,[[Contents Preface University of Hawai‘i at ...,1


In [20]:
# splitting each chunk into its own item

import re

pages_and_chunks = []

for item in tqdm(pages_and_texts):
  for sentence_c in item['sentence_chunks']:
    chunk_dict = {}

    chunk_dict['page_number'] = item['page_number']

    joined_sent_chunk = "".join(sentence_c).replace("  ", " ").strip()
    joined_sent_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sent_chunk)

    chunk_dict['sentence_chunk'] = joined_sent_chunk

    chunk_dict['chunk_char_count'] = len(joined_sent_chunk)

    chunk_dict['chunk_word_count'] = len([word for word in joined_sent_chunk.split(" ")])
    chunk_dict['chunk_token_count'] = len(joined_sent_chunk) / 4

    pages_and_chunks.append(chunk_dict)

  0%|          | 0/1208 [00:00<?, ?it/s]

In [21]:
len(pages_and_chunks)

1843

In [22]:
random.sample(pages_and_chunks, 1)

[{'page_number': 841,
  'sentence_chunk': 'Age Human Milk or infant formula Grain products Vegetables Fruit Protein-rich foods Birth – 6 months Newborns breastfeed 8-12 times/day. Formula fed infants should consume 2-3 ounces of formula every 3-4 hours and by 6 months consume 32 ounces/day. None None None None 6-8 months Breastfed infants continue to breastfeed, on demand. Formula-fed infants take in about 24-32 ounces. Amounts vary based on individual assessment. Intake of human milk or formula may decrease as complementary foods increase. About 1-2 ounces iron-fortified infant cereals, bread, small pieces of crackers About 2-4 ounces of cooked, plain, strained/ pureed/ mash vegetables About 2-4 ounces of plain strained/ pureed/ mashed fruits About 1-2 ounces meat, poultry, fish, eggs, cheese, yogurt, or legumes; all are plain strained/ pureed/ mashed Infancy | 841',
  'chunk_char_count': 835,
  'chunk_word_count': 127,
  'chunk_token_count': 208.75}]

In [23]:
df = pd.DataFrame(pages_and_chunks)

df.describe()

,page_number,chunk_char_count,chunk_word_count,chunk_token_count
count,1843.000000,1843.000000,1843.000000,1843.000000
mean,583.381443,734.829626,112.720022,183.707406
std,347.788670,447.431540,71.065283,111.857885
min,-41.000000,12.000000,3.000000,3.000000
25%,280.500000,315.000000,45.000000,78.750000
50%,586.000000,746.000000,114.000000,186.500000
75%,890.000000,1118.500000,173.000000,279.625000
max,1166.000000,1831.000000,297.000000,457.750000


In [24]:
# filter out chunks of text for short chunks

min_token_len = 30

for row in df[df['chunk_token_count'] <= min_token_len].sample(5).iterrows():
  print(f"Chunk token count : {row[1]['chunk_token_count']} | Text : {row[1]['sentence_chunk']}")

Chunk token count : 3.5 | Text : Fluoride | 697
Chunk token count : 21.0 | Text : Centers for Disease Control and Prevention.http:/ /www.cdc.gov/nutrition/ Iron | 661
Chunk token count : 29.25 | Text : 2010). EH. Net Encyclopedia. http:/ /eh.net/?s=History+of+Food+and+Drug+Regulatio Protecting the Public Health | 1011
Chunk token count : 18.25 | Text : http:/ /pressbooks.oer.hawaii.edu/ humannutrition2/?p=364   630 | Calcium
Chunk token count : 5.25 | Text : 754 | MyPlate Planner


In [25]:
pages_and_chunks_over_min_token_len = df[df['chunk_token_count'] > min_token_len].to_dict(orient = "records")

pages_and_chunks_over_min_token_len[:2]



[{'page_number': -39,
  'sentence_chunk': 'Human Nutrition: 2020 Edition UNIVERSITY OF HAWAI‘I AT MĀNOA FOOD SCIENCE AND HUMAN NUTRITION PROGRAM ALAN TITCHENAL, SKYLAR HARA, NOEMI ARCEO CAACBAY, WILLIAM MEINKE-LAU, YA-YUN YANG, MARIE KAINOA FIALKOWSKI REVILLA, JENNIFER DRAPER, GEMADY LANGFELDER, CHERYL GIBBY, CHYNA NICOLE CHUN, AND ALLISON CALABRESE',
  'chunk_char_count': 308,
  'chunk_word_count': 42,
  'chunk_token_count': 77.0},
 {'page_number': -38,
  'sentence_chunk': 'Human Nutrition: 2020 Edition by University of Hawai‘i at Mānoa Food Science and Human Nutrition Program is licensed under a Creative Commons Attribution 4.0 International License, except where otherwise noted.',
  'chunk_char_count': 210,
  'chunk_word_count': 30,
  'chunk_token_count': 52.5}]

In [26]:
random.sample(pages_and_chunks_over_min_token_len, 1)

[{'page_number': 308,
  'sentence_chunk': 'Soybean oil contains high amounts of polyunsaturated fatty acids. Both 1.\xa0Mori T, Kondo H. (2007). Dietary fish oil upregulates intestinal lipid metabolism and reduces body weight gain in C57BL/6J mice. Journal of Nutrition,\xa0137(12):2629-34. http:/ /www.ncbi.nlm.nih.gov/pubmed/18029475. Accessed September 22, 2017. 308 | How Lipids Work',
  'chunk_char_count': 338,
  'chunk_word_count': 44,
  'chunk_token_count': 84.5}]

In [27]:
## Embedding text chunks
# convert it into meaningful numerical representations
!pip install sentence_transformers


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [28]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer(model_name_or_path = "all-mpnet-base-v2",
                                       device = 'cpu')

sentences = ["The is first sentence",
             "I love tensorflow",
             "Tensorflow is used to create deep learning models"]

embeddings = embedding_model.encode(sentences)

embeddings_dict = dict(zip(sentences, embeddings))

for s, e in embeddings_dict.items():
  print(f"Sentence : {s}")
  print(f"Embedding: {e}")
  print("")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Sentence : The is first sentence
Embedding: [ 7.31820092e-02 -1.61321536e-02 -1.15740197e-02  2.08527446e-02
 -6.85733557e-02  4.06724364e-02 -2.10723169e-02  2.47126687e-02
 -4.70292680e-02 -7.26543739e-03  1.52472192e-02 -3.00663076e-02
  6.55668154e-02 -2.35591270e-02  1.30877784e-02 -2.87996121e-02
  1.62504558e-02  2.79548187e-02  2.96473727e-02 -6.83668815e-03
  2.95617208e-02  3.22184414e-02  1.40264733e-02  1.94812883e-02
  7.42335152e-03 -1.20363701e-02  2.05279533e-02  4.25696336e-02
 -1.84285529e-02 -5.15345521e-02  9.57240351e-03  1.14023760e-02
 -2.68759672e-03 -3.12451944e-02  1.76468416e-06 -8.07849038e-03
 -3.66575941e-02 -3.02734561e-02 -1.20519511e-02  6.74865514e-05
 -2.28438340e-02  5.51525652e-02  1.17068607e-02  3.52899469e-02
 -1.90833863e-02  2.74438281e-02  2.06340905e-02  6.54786825e-02
 -1.78782362e-02  2.59095263e-02 -2.16650646e-02 -1.81301516e-02
 -4.60181050e-02 -1.37205590e-02  9.41626058e-06 -2.37068161e-02
  2.66071297e-02 -3.42558250e-02 -4.00836021e-

In [29]:
len(embeddings[0])

768

In [30]:
pages_and_chunks_over_min_token_len[:2]

[{'page_number': -39,
  'sentence_chunk': 'Human Nutrition: 2020 Edition UNIVERSITY OF HAWAI‘I AT MĀNOA FOOD SCIENCE AND HUMAN NUTRITION PROGRAM ALAN TITCHENAL, SKYLAR HARA, NOEMI ARCEO CAACBAY, WILLIAM MEINKE-LAU, YA-YUN YANG, MARIE KAINOA FIALKOWSKI REVILLA, JENNIFER DRAPER, GEMADY LANGFELDER, CHERYL GIBBY, CHYNA NICOLE CHUN, AND ALLISON CALABRESE',
  'chunk_char_count': 308,
  'chunk_word_count': 42,
  'chunk_token_count': 77.0},
 {'page_number': -38,
  'sentence_chunk': 'Human Nutrition: 2020 Edition by University of Hawai‘i at Mānoa Food Science and Human Nutrition Program is licensed under a Creative Commons Attribution 4.0 International License, except where otherwise noted.',
  'chunk_char_count': 210,
  'chunk_word_count': 30,
  'chunk_token_count': 52.5}]

In [31]:
# %%time

# embedding_model.to('cpu')

# for i in tqdm(pages_and_chunks_over_min_token_len):
#   i['embedding'] = embedding_model.encode(i['sentence_chunk'])

In [32]:
%%time
embedding_model.to('cuda')

for i in tqdm(pages_and_chunks_over_min_token_len):
  i['embedding'] = embedding_model.encode(i['sentence_chunk'])

  0%|          | 0/1680 [00:00<?, ?it/s]

CPU times: user 34.8 s, sys: 453 ms, total: 35.3 s
Wall time: 42.4 s


In [33]:
%%time

text_chunks = [i['sentence_chunk'] for i in pages_and_chunks_over_min_token_len]

text_chunks[419]

CPU times: user 483 µs, sys: 0 ns, total: 483 µs
Wall time: 490 µs


'often. • Calm your “sweet tooth” by eating fruits, such as berries or an apple. • Replace sugary soft drinks with seltzer water, tea, or a small amount of 100 percent fruit juice added to water or soda water. The Food Industry: Functional Attributes of Carbohydrates and the Use of Sugar Substitutes In the food industry, both fast-releasing and slow-releasing carbohydrates are utilized to give foods a wide spectrum of functional attributes, including increased sweetness, viscosity, bulk, coating ability, solubility, consistency, texture, body, and browning capacity. The differences in chemical structure between the different carbohydrates confer their varied functional uses in foods. Starches, gums, and pectins are used as thickening agents in making jam, cakes, cookies, noodles, canned products, imitation cheeses, and a variety of other foods. Molecular gastronomists use slow- releasing carbohydrates, such as alginate, to give shape and texture to their fascinating food creations. Add

In [34]:
len(text_chunks)

1680

In [35]:
%%time

text_chunk_embeddings = embedding_model.encode(text_chunks,
                                               batch_size = 32,
                                               convert_to_tensor = True)

text_chunk_embeddings



CPU times: user 23.8 s, sys: 75.3 ms, total: 23.9 s
Wall time: 23.5 s


tensor([[ 0.0674,  0.0902, -0.0051,  ..., -0.0221, -0.0232,  0.0126],
        [ 0.0552,  0.0592, -0.0166,  ..., -0.0120, -0.0103,  0.0227],
        [ 0.0280,  0.0340, -0.0206,  ..., -0.0054,  0.0213,  0.0313],
        ...,
        [ 0.0771,  0.0098, -0.0122,  ..., -0.0409, -0.0752, -0.0241],
        [ 0.1030, -0.0165,  0.0083,  ..., -0.0574, -0.0283, -0.0295],
        [ 0.0864, -0.0125, -0.0113,  ..., -0.0522, -0.0337, -0.0299]],
       device='cuda:0')

In [36]:
text_chunks_and_embeddings_df = pd.DataFrame(pages_and_chunks_over_min_token_len)

embeddings_df_save_path = 'text_chunks_and_embeddings_df.csv'
text_chunks_and_embeddings_df.to_csv(embeddings_df_save_path, index = False)

In [37]:
text_embeddings_df_load = pd.read_csv(embeddings_df_save_path)
text_embeddings_df_load.head()

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,-39,Human Nutrition: 2020 Edition UNIVERSITY OF HA...,308,42,77.00,[ 6.74242675e-02 9.02281404e-02 -5.09548886e-...
1,-38,Human Nutrition: 2020 Edition by University of...,210,30,52.50,[ 5.52156419e-02 5.92139773e-02 -1.66167244e-...
2,-37,Contents Preface University of Hawai‘i at Māno...,766,114,191.50,[ 2.79801842e-02 3.39813754e-02 -2.06426680e-...
3,-36,Lifestyles and Nutrition University of Hawai‘i...,941,142,235.25,[ 6.82566911e-02 3.81275006e-02 -8.46854132e-...
4,-35,The Cardiovascular System University of Hawai‘...,998,152,249.50,[ 3.30264494e-02 -8.49763490e-03 9.57159605e-...


In [56]:
import random
import torch
import numpy as np
import pandas as pd

device = 'cuda' if torch.cuda.is_available() else 'cpu'

text_chunks_embedding_df = pd.read_csv("text_chunks_and_embeddings_df.csv")

text_chunks_embedding_df['embedding'] = text_chunks_embedding_df['embedding'].apply(lambda x: np.fromstring(x.strip('[]'), sep = " "))


embeddings = torch.tensor(np.stack(text_chunks_embedding_df['embedding'].tolist(), axis = 0), dtype = torch.float32).to(device)

pages_and_chunks = text_chunks_and_embeddings_df.to_dict(orient = 'records')



In [57]:
text_chunks_embedding_df.head()

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,-39,Human Nutrition: 2020 Edition UNIVERSITY OF HA...,308,42,77.00,"[0.0674242675, 0.0902281404, -0.00509548886, -..."
1,-38,Human Nutrition: 2020 Edition by University of...,210,30,52.50,"[0.0552156419, 0.0592139773, -0.0166167244, -0..."
2,-37,Contents Preface University of Hawai‘i at Māno...,766,114,191.50,"[0.0279801842, 0.0339813754, -0.020642668, 0.0..."
3,-36,Lifestyles and Nutrition University of Hawai‘i...,941,142,235.25,"[0.0682566911, 0.0381275006, -0.00846854132, -..."
4,-35,The Cardiovascular System University of Hawai‘...,998,152,249.50,"[0.0330264494, -0.0084976349, 0.00957159605, -..."


In [58]:
# embeddings = text_chunks_embedding_df['embedding'].tolist()
# embeddings
embeddings

tensor([[ 0.0674,  0.0902, -0.0051,  ..., -0.0221, -0.0232,  0.0126],
        [ 0.0552,  0.0592, -0.0166,  ..., -0.0120, -0.0103,  0.0227],
        [ 0.0280,  0.0340, -0.0206,  ..., -0.0054,  0.0213,  0.0313],
        ...,
        [ 0.0771,  0.0098, -0.0122,  ..., -0.0409, -0.0752, -0.0241],
        [ 0.1030, -0.0165,  0.0083,  ..., -0.0574, -0.0283, -0.0295],
        [ 0.0864, -0.0125, -0.0113,  ..., -0.0522, -0.0337, -0.0299]],
       device='cuda:0')

In [59]:
embeddings.shape

torch.Size([1680, 768])

In [60]:
from sentence_transformers import util, SentenceTransformer

embedding_model = SentenceTransformer(model_name_or_path='all-mpnet-base-v2',
                                      device=device)

###Embedding model ready
Small semantic search pipeline

In [65]:
query = 'breastfeeding timeline for infants'
print(f'Query : {query}')


query_embedding = embedding_model.encode(query, convert_to_tensor = True).to('cuda')


from time import perf_counter as timer
start_time = timer()
dot_scores = util.dot_score(query_embedding, embeddings)[0]
end_time = timer()

print(f"Time taken to get score on {len(embeddings)} embeddings : {end_time - start_time:.5f} seconds")

print("")

top_results_ = torch.topk(dot_scores, 5)

top_results_

Query : breastfeeding timeline for infants
Time taken to get score on 1680 embeddings : 0.00022 seconds



torch.return_types.topk(
values=tensor([0.6749, 0.6574, 0.6543, 0.6354, 0.6168], device='cuda:0'),
indices=tensor([1151, 1169, 1160, 1144, 1138], device='cuda:0'))

In [71]:
pages_and_chunks[1151]['sentence_chunk'], pages_and_chunks[1151]['page_number']

('milk is the best source to fulfill nutritional requirements. An exclusively breastfed infant does not even need extra water, including in hot climates. A newborn infant (birth to 28 days) requires feedings eight to twelve times a day or more. Between 1 and 3 months of age, the breastfed infant becomes more efficient, and the number of feedings per day often become fewer even though the amount of milk consumed stays the same. After about six months, infants can gradually begin to consume solid foods to help meet nutrient needs. Foods that are added in addition to breastmilk are called complementary foods. Complementary foods should be nutrient dense to provide optimal nutrition. Complementary foods include baby meats, vegetables, fruits, infant cereal, and dairy products such as yogurt, but not infant formula. Infant formula is a substitute, not a complement to breastmilk. In addition to complementary foods, the World Health Organization recommends that breastfeeding continue up to 2 

In [73]:
larger_embeddings = torch.randn(1000*embeddings.shape[0], 768).to(device)

print(f'Embeddings shape: {larger_embeddings.shape}')

start_time = timer()
dot_scores = util.dot_score(query_embedding, larger_embeddings)[0]
end_time = timer()



print(f"Time taken to get score on {len(larger_embeddings)} embeddings : {end_time - start_time:.5f} seconds")

print("")



Embeddings shape: torch.Size([1680000, 768])
Time taken to get score on 1680000 embeddings : 0.00120 seconds

